In [16]:
import pandas as pd
from pandasql import sqldf
import numpy as np

In [17]:
data = pd.read_csv("../data/For Students-Updated2022-03-25/data files/logs.csv")

/tmp/ipykernel_101776/2077613250.py:1: DtypeWarning: Columns (34,38,39,40,47,50,53,54,57,58,59,60,62,68,70,72,73,74,76,77,78,81,85,88,89,90,91,92,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,118,120,121,122,123,129,131) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/For Students-Updated2022-03-25/data files/logs.csv")


In [18]:
swear_words = list(pd.read_csv("../data/external/bad_words.csv").en_bad_words)

In [19]:
#features = ["idle_time_60", "idle_count_60", "stars_by_minigames_completed", "events_in_session", "profanity", "total_quits", "proportion_filled_labels"]

In [20]:
# Idling Calculations
def idle_time(data, idle_period=60):
    idle_time = 0
    idle_instances = 0
    for i, x in data.iterrows():
        if i!=0:
            diff=x.event_time_dbl-data.iloc[i-1].event_time_dbl
            if diff>idle_period:
                idle_time+=diff
                idle_instances+=1
    return (idle_time, idle_instances)

def full_idle(data, idle_period=60):
    results = {
        'player_id':[],
        f'idle_time_{idle_period}':[],
        f'idle_count_{idle_period}':[]
    }
    
    temp = data.copy()
    temp.reset_index(inplace=True)
    results['player_id'].append(temp.player_id.unique()[0])
    time, instance = idle_time(temp, idle_period=idle_period)
    results[f'idle_time_{idle_period}'].append(time)
    results[f'idle_count_{idle_period}'].append(instance)
    
    return pd.DataFrame(results)

In [21]:
def count_events(data):
    return pd.DataFrame({"player_id":list(data.player_id)[0],"total_events":[data.shape[0]]})

def total_time(data):
    init_time = list(data["event_time_dbl"])[0]
    end_time = list(data["event_time_dbl"])[-1]
    return pd.DataFrame({"player_id":list(data.player_id)[0],"total_time":[end_time-init_time]})

In [22]:
def used_profanity(data):
    profanity_count = 0
    profanity = ""
    for i in " ".join(list(data.new_label.dropna().unique())).split(" "):
        if(i in swear_words):
            profanity_count += 1
            profanity += i + ", "
    if(profanity != ""):
        profanity = profanity[:-2]
    return pd.DataFrame({"player_id":list(data.player_id)[0], "profanity_uses":[profanity_count], "profane_text":[profanity if profanity != "" else np.nan]})

In [23]:
def get_quit_count(data):
    num_quits = data[data["event_id"] == 1002].shape[0]
    return pd.DataFrame({"player_id":list(data.player_id)[0],"total_quits":[num_quits]})

In [24]:
def get_labels(data):
    available_labels = ['020', '820', '350', '430', '121', '021', '521', 
                '320', '300', '830', '630', '810', '120', '011', 
                '010', '420', '640', '210', '110', '840', '022', 
                '111', '520', '200', '540']
    appeared_labels = []
    filled_labels = []
    
    for i in available_labels:
        for index, row in data[(data["aa_level_id"] == int(i[0]))&(data["chunk_id"] == int(i[1]))&(data["piece_id"] == int(i[0]))].iterrows():
            if(i not in filled_labels and row["new_label"] is not np.nan):
                if(i not in appeared_labels):
                    appeared_labels.append(i)
                filled_labels.append(i)
            elif(i not in appeared_labels and row["new_label"] is np.nan):
                appeared_labels.append(i)
    if(len(appeared_labels) == 0):
        prop = 1
    else:
        prop = len(filled_labels)/len(appeared_labels)
    return pd.DataFrame({"player_id":list(data.player_id)[0],"label_filled_count":[len(filled_labels)],"proportion_filled_labels":[prop]})

In [25]:
def get_minigames_completed(data):
    completed = 0
    total_done = data[(data["event_id"] == 1001)].shape[0]
    total_restart = data[(data["event_id"] == 1000)].shape[0]
    return pd.DataFrame({"player_id":list(data.player_id)[0],"total_minigames_done_btn":[total_done],"total_restarts":[total_restart]})

In [26]:
def total_stars_earned(data):
    player_ids = data.player_id.unique()
    results = {'player_id':[],'total_stars':[]}
    data = data[data.event_id == 1005]
    for p_id in player_ids:
        temp_data = data[data.player_id==p_id]
        stars = 0
        for i, x in data.iterrows():
            stars += x.new_skill_point - x.old_skill_point
        results['player_id'].append(p_id)
        results['total_stars'].append(stars)
    return pd.DataFrame(results)

In [27]:
def unique_minigames_completed(sessions):
    completed_minigames = set()
    new = {}
    for s in sessions:
        this_set = set()
        data = sessions[s][(sessions[s].event_id==1001)|(sessions[s].event_id==1003)].reset_index(drop=True)
        last_id = -1
        for i, x in data.iterrows():
            if x.event_id == 1003:
                last_id = x.minigame_id
            elif x.event_id == 1001:
                this_set.add((last_id, x.minigame_level))
        try:
            pid = list(data.iloc[0].player_id)[0]
        except:
            pid = np.nan
        new[s] = pd.DataFrame({
            'player_id': [pid],
            'unique_minigames_completed': [len(this_set-completed_minigames)]
        })
        completed_minigames = completed_minigames.union(this_set)
    return new

In [ ]:
import pickle
test_data = data.copy()
ex_subset = None
players = {x:[] for x in test_data.player_id.unique()}
for i,player in enumerate(test_data.player_id.unique()):
    try:
        p_subset = test_data[test_data["player_id"] == player]
        sessions = {x:[] for x in p_subset.session.unique()}
        for session in p_subset.session.unique():

            s_subset = p_subset[p_subset["session"] == session]

            full_sub = s_subset.copy()

            s_idle = full_idle(s_subset, idle_period=60)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_idle ON full_sub.player_id = s_idle.player_id")
            s_idle = full_idle(s_subset, idle_period=300)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_idle ON full_sub.player_id = s_idle.player_id")
            s_event_count = count_events(s_subset)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_event_count ON full_sub.player_id = s_event_count.player_id")
            s_total_time = total_time(s_subset)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_total_time ON full_sub.player_id = s_total_time.player_id")
            s_profanity = used_profanity(s_subset)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_profanity ON full_sub.player_id = s_profanity.player_id")
            s_quit = get_quit_count(s_subset)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_quit ON full_sub.player_id = s_quit.player_id")
            s_labels = get_labels(s_subset)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_labels ON full_sub.player_id = s_labels.player_id")
            s_minigames = get_minigames_completed(s_subset)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_minigames ON full_sub.player_id = s_minigames.player_id")
            s_stars = total_stars_earned(s_subset)
            full_sub = sqldf("SELECT * FROM full_sub INNER JOIN s_stars ON full_sub.player_id = s_stars.player_id")

            print(session)
            if(ex_subset is None):
                ex_subset = full_sub

            sessions[session] = full_sub
        p_session_mgs = unique_minigames_completed(sessions)
        for i in p_session_mgs:
            session_df = sessions[i].T.drop_duplicates().T
            #print(session_df)
            session_mgs_df = p_session_mgs[i].T.drop_duplicates().T
            #print(session_mgs_df.player_id)
            #print(session_mgs_df.unique_minigames_completed)
            sessions[i] = sqldf("SELECT * FROM session_df INNER JOIN session_mgs_df ON session_df.player_id = session_mgs_df.player_id")
        players[player] = sessions

        output = open('../data/generated/player_by_session.pkl', 'wb')
        pickle.dump(players, output)
        output.close()
    except Exception:pass
#players

Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 8
Session 9
Session 10
Session 11
Session Control Group
Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 8
Session 9
Session Control Group
Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 8
Session 9
Session 10
Session Control Group
Session 1
Session 2
Session 4
Session 5
Session 6
Session 7
Session 1
Session 2
Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 8
Session 9
Session 1
Session 2
Session 3
Session 1
Session 2
Session 3
Session 4
Session 5
Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 8
Session 9
Session 1
Session 2
Session 4
Session 5
Session 6
Session 7
Session 8
Session 9
Session 10
Session 11
Session 1
Session 2
Session 3
Session 4
Session 5

Session 3
Session 4
Session 5
Session 6
Session 7
Session 8
Session 1
Session 2
Session 3
Session 4
Session 5
Session 1
Session 2
Session 3
Session 4
Session 5
Session 6
Session 7
Session 8
Session 10
Session 9
Sessions 1_ 2
Sessions 3 _ 4
Sessions 5 _ 6
Sessions 7 _ 8
Session 9
Session 10
Session 11
Session 12
Sessions 1 _ 2
Sessions 3 _ 4
Sessions 5 _ 6
Sessions 7 _ 8
Session 9
Session 10
Session 11
Session 12
Session 13
Sessions 1 _ 2
Sessions 3 _ 4
Session 5
Sessions 3 _ 4
Sessions 5 _ 6
Session 7
Session 8
Session 9
Session 10
Sessions 1 _ 2
Sessions 3 _ 4
Session 5
Session 6
Session 7
Session 8
Session 9
Session 10
Session 11
Sessions 1 _ 2
Session 3 _ 4
Sessions 5 _ 6
Sessions 7 _ 8
Session 9
Session 10
Session 11
Session 12
Session 13
Sessions 1 _ 2
Session 3 _ 4
Sessions 5 _ 6
Sessions 7 _ 8
Session 9
Session 10
Session 11
Session 12
Session 13
Session 1 _ 2
Sessions 3 _ 4
Sessions 5 _ 6
Session 7
Session 8
Session 9
Session 10
Session 11
Sessions 1 _ 2
Session 3 _ 4
Sessions 

Exception ignored in: <function _collection_gced at 0x7fbcc0bb4310>
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.9/site-packages/sqlalchemy/event/registry.py", line 53, in _collection_gced
    def _collection_gced(ref):
KeyboardInterrupt: 


Session 1
Session 6
Session 7
Session 8
Session 9
Session 9


In [ ]:
players[list(players.keys())[0]]["Sessions 1_ 2"]

In [ ]:
import dtale
d = dtale.show(sqldf("SELECT * FROM ex_subset GROUP BY player_id"))
print(d._main_url)

In [ ]:
players

In [ ]:
#import pickle
#output = open('../data/generated/player_by_session_ex.pkl', 'wb')
#pickle.dump(players, output)
#output.close()